<a href="https://colab.research.google.com/github/rytzi/Durian-Ripeness-Detection-Model/blob/main/CNN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
import os
import cv2
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers, models

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
data_dir = '/content/drive/MyDrive/Durian Training Dataset'
classes = ['Ripe', 'Unripe']
images = []
labels = []

for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    group_folders = os.listdir(class_dir)

    for group_folder in group_folders:
        group_dir = os.path.join(class_dir, group_folder)
        image_names = os.listdir(group_dir)

        for i in range(min(4, len(image_names))):
            image_name = image_names[i]
            image_path = os.path.join(group_dir, image_name)
            image = cv2.imread(image_path)
            images.append(image)
            labels.append(class_label)


In [ ]:
from keras.applications import VGG16
from keras import models
from keras import layers

image_arrays_np = [np.resize(img, (100, 100, 3)) for img in images]

X = np.array(image_arrays_np)
y = np.array(labels)
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=2)

vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

for layer in vgg_base.layers:
    layer.trainable = False

model = models.Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y_one_hot, epochs=10, batch_size=10)
_, accuracy = model.evaluate(X, y_one_hot)
print('Accuracy: %.2f' % (accuracy*100))


model.save("durian_image_model.keras")

Epoch 1/10
10/10 [==============================] - 12s 1s/step - loss: 26.0924 - accuracy: 0.5652
Epoch 2/10
10/10 [==============================] - 11s 1s/step - loss: 10.1118 - accuracy: 0.7065
Epoch 3/10
10/10 [==============================] - 10s 1s/step - loss: 2.9446 - accuracy: 0.7826
Epoch 4/10
10/10 [==============================] - 10s 975ms/step - loss: 1.6872 - accuracy: 0.8804
Epoch 5/10
10/10 [==============================] - 11s 1s/step - loss: 0.3933 - accuracy: 0.9022
Epoch 6/10
10/10 [==============================] - 11s 1s/step - loss: 0.1205 - accuracy: 0.9674
Epoch 7/10
10/10 [==============================] - 9s 925ms/step - loss: 0.1105 - accuracy: 0.9565
Epoch 8/10
10/10 [==============================] - 11s 1s/step - loss: 0.0013 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 11s 1s/step - loss: 6.3648e-04 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 12s 4s/step - loss: 5.2006e-04 - accuracy: 1.0000
Accura

In [ ]:
model.save("cnn_durio.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('cnn_model.tflite', 'wb') as f:
    f.write(tflite_model)